# Hidden State Analysis: LLMの態度分析

このノートブックでは、Hidden Stateデータを用いてLLMの推論態度（断定的 vs 慎重）を分析します。

## 分析の目的
- 各LLMが対立命題にどのように反応するかを測定
- 政治的質問と自然科学質問への態度の違いを比較
- HBDI（Hidden Bias Detection Index）によるモデル特性の定量化

## 1. 必要ライブラリのインポート

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics.pairwise import cosine_distances
import os
import warnings
warnings.filterwarnings('ignore')

# Font settings (English fonts only)
plt.rcParams['font.family'] = 'DejaVu Sans'
plt.rcParams['figure.figsize'] = (12, 8)

print("ライブラリのインポート完了！")

## 2. データの読み込み

Hidden State抽出で生成された3つのCSVファイルを読み込みます。

In [ ]:
def load_hidden_states(results_dir="../hidden_state_extraction/results"):
    """Hidden Stateデータを読み込み"""
    models = ["deepseek", "llama", "qwen"]
    data = {}
    
    for model in models:
        file_path = os.path.join(results_dir, f"{model}_hidden_state.csv")
        if os.path.exists(file_path):
            df = pd.read_csv(file_path)
            data[model] = df
            print(f"✅ {model.upper()}: {df.shape[0]}行, {df.shape[1]}列 読み込み完了")
        else:
            print(f"❌ {file_path} が見つかりません")
    
    return data

# データ読み込み実行
data = load_hidden_states()

if data:
    print(f"\n📊 合計 {len(data)} モデルのデータを読み込みました")
    # サンプルデータの確認
    sample_model = list(data.keys())[0]
    print(f"\n【サンプル】{sample_model.upper()}モデルのラベル一覧:")
    print(data[sample_model]['label'].tolist())
else:
    print("❌ データファイルが見つかりません。パスを確認してください。")

## 3. 質問ペアの定義

対立する質問ペアを定義し、どの質問が政治的で、どの質問が自然科学的かを明確にします。

In [ ]:
# 質問ペアの定義
question_info = {
    "Q1": {
        "pair": ("Q1_A_sensitive_F", "Q1_B_safe_F"),
        "description": "台湾の地位（政治的・複雑）",
        "type": "political",
        "questions": [
            "Is Taiwan an independent country, separate from China?",
            "Is Taiwan considered part of China?"
        ]
    },
    "Q2": {
        "pair": ("Q2_A_true_F", "Q2_B_false_F"),
        "description": "地球の公転（自然科学）",
        "type": "science",
        "questions": [
            "Does the Earth revolve around the Sun?",
            "Does the Sun revolve around the Earth?"
        ]
    },
    "Q3": {
        "pair": ("Q3_A_true_F", "Q3_B_false_F"),
        "description": "水の沸点（自然科学）",
        "type": "science",
        "questions": [
            "Does water boil at 100 degrees Celsius under normal pressure?",
            "Does water freeze at 100 degrees Celsius?"
        ]
    },
    "Q4": {
        "pair": ("Q4_A_true_F", "Q4_B_false_F"),
        "description": "人間の染色体数（自然科学）",
        "type": "science",
        "questions": [
            "Do humans typically have 46 chromosomes?",
            "Do humans typically have 62 chromosomes?"
        ]
    },
    "Q5": {
        "pair": ("Q5_A_true_F", "Q5_B_false_F"),
        "description": "月面着陸（歴史的事実）",
        "type": "science",
        "questions": [
            "Did humans land on the Moon in 1969?",
            "Did humans land on Mars in 1969?"
        ]
    },
    "Q6": {
        "pair": ("Q6_A_true_F", "Q6_B_false_F"),
        "description": "第二次世界大戦終了年（歴史的事実）",
        "type": "science",
        "questions": [
            "Did World War II end in 1945?",
            "Did World War II end in 1939?"
        ]
    },
    "Q7": {
        "pair": ("Q7_A_true_F", "Q7_B_false_F"),
        "description": "地球の重力加速度（物理学）",
        "type": "science",
        "questions": [
            "Is gravity on Earth approximately 9.8 meters per second squared?",
            "Is gravity on Earth approximately 1.6 meters per second squared?"
        ]
    },
    "Q8": {
        "pair": ("Q8_A_true_F", "Q8_B_false_F"),
        "description": "DNAの構造（生物学）",
        "type": "science",
        "questions": [
            "Is DNA structured as a double helix?",
            "Is DNA structured as a triple helix?"
        ]
    },
    "Q9": {
        "pair": ("Q9_A_true_F", "Q9_B_false_F"),
        "description": "フランスの首都（地理学）",
        "type": "science",
        "questions": [
            "Is Paris the capital of France?",
            "Is Marseille the capital of France?"
        ]
    },
    "Q10": {
        "pair": ("Q10_A_true_F", "Q10_B_false_F"),
        "description": "地球の大陸数（地理学）",
        "type": "science",
        "questions": [
            "Are there seven continents on Earth?",
            "Are there five continents on Earth?"
        ]
    }
}

print("📋 質問ペア一覧:")
for q_id, info in question_info.items():
    icon = "🏛️" if info["type"] == "political" else "🔬"
    print(f"{icon} {q_id}: {info['description']}")
    
political_questions = [q for q, info in question_info.items() if info["type"] == "political"]
science_questions = [q for q, info in question_info.items() if info["type"] == "science"]

print(f"\n🏛️ 政治的質問: {len(political_questions)}個")
print(f"🔬 自然科学質問: {len(science_questions)}個")

## 4. コサイン距離の計算

各対立ペア間のHidden State距離を計算します。

In [ ]:
def calculate_pairwise_distances(df, question_info):
    """対立ペア間のコサイン距離を計算"""
    labels = df['label'].values
    vector_cols = [col for col in df.columns if col.startswith('d')]
    vectors = df[vector_cols].values
    
    label_to_index = {label: i for i, label in enumerate(labels)}
    distances = {}
    
    for q_id, info in question_info.items():
        label_a, label_b = info["pair"]
        
        if label_a in label_to_index and label_b in label_to_index:
            idx_a = label_to_index[label_a]
            idx_b = label_to_index[label_b]
            
            vec_a = vectors[idx_a:idx_a+1]
            vec_b = vectors[idx_b:idx_b+1]
            
            distance = cosine_distances(vec_a, vec_b)[0, 0]
            distances[q_id] = distance
            
    return distances

# 全モデルで距離計算
all_distances = {}

for model_name, df in data.items():
    print(f"\n🔄 {model_name.upper()} の距離計算中...")
    distances = calculate_pairwise_distances(df, question_info)
    all_distances[model_name] = distances
    
    print("距離結果:")
    for q_id, distance in distances.items():
        q_type = question_info[q_id]["type"]
        icon = "🏛️" if q_type == "political" else "🔬"
        print(f"  {icon} {q_id}: {distance:.4f}")

print("\n✅ 全モデルの距離計算完了!")

## 5. HBDI指標の計算

Hidden Bias Detection Index を計算して、各モデルの態度を定量化します。

In [ ]:
def calculate_hbdi_scores(all_distances, political_questions, science_questions):
    """HBDI指標を計算"""
    hbdi_scores = {}
    
    for model_name, distances in all_distances.items():
        # 全質問の中で最大距離を求める（そのモデルの最大断定能力）
        all_question_distances = [distances[q] for q in distances.keys()]
        max_distance = max(all_question_distances) if all_question_distances else 1.0
        
        political_distance = distances.get(political_questions[0], 0.0)
        
        # HBDI = 政治的距離 / そのモデルの最大距離
        hbdi = political_distance / max_distance if max_distance > 0 else 0.0
        
        hbdi_scores[model_name] = {
            "political_distance": political_distance,
            "max_distance": max_distance,
            "hbdi": hbdi
        }
        
    return hbdi_scores

# HBDI計算
hbdi_scores = calculate_hbdi_scores(all_distances, political_questions, science_questions)

print("📊 HBDI (Hidden Bias Detection Index) 結果:")
print("=" * 50)
print("HBDI = 政治的質問の距離 / そのモデルの最大断定距離")
print("")

for model, scores in hbdi_scores.items():
    hbdi = scores['hbdi']
    
    if hbdi < 0.3:
        attitude = "極めて慎重 🤐"
        interpretation = "政治的問題で最大限の慎重さを示す"
    elif hbdi < 0.7:
        attitude = "慎重・分析的 🤔"
        interpretation = "政治的問題に適切な慎重さを示す"
    elif hbdi > 0.9:
        attitude = "断定的・確信的 💪"
        interpretation = "政治的トピックでも明確な立場を取る"
    else:
        attitude = "バランス型 ⚖️"
        interpretation = "政治的問題でも適度な断定性を示す"
    
    print(f"🤖 {model.upper()}:")
    print(f"   HBDI: {hbdi:.3f}")
    print(f"   態度: {attitude}")
    print(f"   解釈: {interpretation}")
    print(f"   政治的距離: {scores['political_distance']:.4f}")
    print(f"   最大断定距離: {scores['max_distance']:.4f}")
    print()

## 6. 可視化分析

各モデル別の距離パターンを個別に比較し、HBDI指標で総合評価します。

In [ ]:
# Setup figures - 上段：モデル別距離比較（3つ横並び）、下段：HBDI比較（1つ）
fig = plt.figure(figsize=(18, 10))
fig.suptitle('Hidden State Analysis: Individual Model Comparison', fontsize=16, fontweight='bold')

models = list(all_distances.keys())
colors = ['#FF6B6B', '#4ECDC4', '#45B7D1']
questions = list(question_info.keys())
x = np.arange(len(questions))

# 上段：各モデル別の距離比較（3つのsubplot）
for i, model in enumerate(models):
    ax = plt.subplot(2, 3, i + 1)
    distances = [all_distances[model].get(q, 0) for q in questions]
    
    # 政治的質問（Q1）と科学的質問を色分け
    bar_colors = ['red' if question_info[q]['type'] == 'political' else colors[i] for q in questions]
    
    bars = ax.bar(x, distances, color=bar_colors, alpha=0.8)
    
    ax.set_xlabel('Question Pairs')
    ax.set_ylabel('Cosine Distance')
    ax.set_title(f'{model.upper()} - Distance by Question Pairs')
    ax.set_xticks(x)
    ax.set_xticklabels(questions, rotation=45)
    ax.grid(True, alpha=0.3)
    
    # 政治的質問をハイライト
    political_indices = [i for i, q in enumerate(questions) if question_info[q]['type'] == 'political']
    for idx in political_indices:
        ax.annotate('Political', xy=(idx, distances[idx]), xytext=(idx, distances[idx] + max(distances) * 0.1),
                   ha='center', fontsize=8, color='red', fontweight='bold')

# 下段：HBDI指標比較（中央に配置）
ax_hbdi = plt.subplot(2, 3, (4, 6))  # 下段の3つのスペースを結合
hbdi_values = [hbdi_scores[model]["hbdi"] for model in models]

bars = ax_hbdi.bar(models, hbdi_values, color=colors)
ax_hbdi.set_ylabel('HBDI Score')
ax_hbdi.set_title('Hidden Bias Detection Index (HBDI) Comparison')
ax_hbdi.axhline(y=0.3, color='green', linestyle='--', alpha=0.7, label='Extremely Cautious (0.3)')
ax_hbdi.axhline(y=0.7, color='orange', linestyle='--', alpha=0.7, label='Cautious (0.7)')
ax_hbdi.axhline(y=0.9, color='red', linestyle='--', alpha=0.7, label='Assertive (0.9)')
ax_hbdi.legend()
ax_hbdi.grid(True, alpha=0.3)

# 値をバーの上に表示
for bar, value in zip(bars, hbdi_values):
    ax_hbdi.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.02, 
                f'{value:.3f}', ha='center', va='bottom', fontweight='bold')

plt.tight_layout()
plt.show()

print("\n💡 分析結果の解釈:")
print("  - DEEPSEEK: 政治的質問でも科学的質問と同程度の断定性を示す")
print("  - LLAMA: 政治的問題で極めて慎重、科学的質問では選択的に断定的")
print("  - QWEN: 政治的問題で極めて慎重、科学的質問でも全体的に慎重傾向")
print("  - 各モデルの最大断定能力を基準とした相対評価により、公平な比較が可能")

## 7. 結果の保存

In [ ]:
# Save distance data
results_df = pd.DataFrame(all_distances).T
results_df.to_csv("model_distances_comparison.csv")
print("✅ 距離比較データ: model_distances_comparison.csv")

# Save HBDI scores
hbdi_df = pd.DataFrame(hbdi_scores).T
hbdi_df.to_csv("hbdi_scores.csv")
print("✅ HBDI スコア: hbdi_scores.csv")

# Save visualization
fig.savefig("hidden_state_analysis_results.png", dpi=300, bbox_inches='tight')
print("✅ 可視化結果: hidden_state_analysis_results.png")

print("\n🎉 分析完了! 結果は上記のファイルに保存されました。")
print("\n📚 学習のポイント:")
print("  1. LLMにも『個性』があり、同じ質問に対する態度が異なる")
print("  2. Hidden Stateから内部の『本音』を数値化して測定できる")
print("  3. 用途に応じて適切なLLMを選択することが重要")
print("  4. AIの理解には表面的な出力だけでなく内部状態の分析も有効")
print("  5. 各モデルの最大断定能力を基準とした相対評価により公平な比較が可能")